In [135]:
metrics_map = {
    "conflicts":                   "conflicts",
    "quints":                      "quints",
    "quads":                       "quads",
    "four in five slots":          "four_in_five",
    "triple in 24h (no gaps)":     "triple_in_24h",
    "triple in same day (no gaps)": "triple_in_same_day",
    "three in four slots":         "three_in_four",
    "evening/morning b2b":         "evening_morning_b2b",
    "other b2b":                   "other_b2b",
    "two in three slots":          "two_in_three",
    "singular late exam":          "singular_late",
    "two exams, large gap":        "two_large_gap",
    "avg_max":                     "avg_max",
    "lateness":                    "lateness",
    "size_cutoff":                 "size_cutoff",
    "reserved":                    "reserved",
    "num_blocks":                  "num_blocks",
    "alpha":                       "alpha",
    "gamma":                       "gamma",
    "delta":                       "delta",
    "vega":                        "vega",
    "theta":                       "theta",
    "large_block_size":            "large_block_size",
    "large_exam_weight":           "large_exam_weight",
    "large_block_weight":          "large_block_weight",
    "large_size_1":                "large_size_1",
    "large_cutoff_freedom":        "large_cutoff_freedom",
    "tradeoff":                    "tradeoff",
    "flpens":                      "flpens",
    "semester" : 'semester'
}

reverse_metrics_map = {v: k for k, v in metrics_map.items()}

col_map = {
    "Exam Group": "exam_id",
    "Exam Block": "block",
    # if your sheets ever include a “Semester” column, you could map it too:
    # "Semester":    "semester"
}
inv_map = {v: k for k, v in col_map.items()}
inv_map["semester"] = "Semester"


# map your CSV’s columns → the DB columns
col_map_schedule = {
    "Exam Group":   "exam_id",
    "slot":      "slot",
    "Semester":  "semester",
    # if your CSV also has a “Faculty” column, you could add:
    # "Faculty":   "faculty",
}
inv_map_schedule = {v: k for k, v in col_map_schedule.items()}

metrics_map = {
    "conflicts":                   "conflicts",
    "quints":                      "quints",
    "quads":                       "quads",
    "four in five slots":          "four_in_five",
    "triple in 24h (no gaps)":     "triple_in_24h",
    "triple in same day (no gaps)": "triple_in_same_day",
    "three in four slots":         "three_in_four",
    "evening/morning b2b":         "evening_morning_b2b",
    "other b2b":                   "other_b2b",
    "two in three slots":          "two_in_three",
    "singular late exam":          "singular_late",
    "two exams, large gap":        "two_large_gap",
    "avg_max":                     "avg_max",
    "lateness":                    "lateness",
    "size_cutoff":                 "size_cutoff",
    "reserved":                    "reserved",
    "num_blocks":                  "num_blocks",
    "alpha":                       "alpha",
    "gamma":                       "gamma",
    "delta":                       "delta",
    "vega":                        "vega",
    "theta":                       "theta",
    "large_block_size":            "large_block_size",
    "large_exam_weight":           "large_exam_weight",
    "large_block_weight":          "large_block_weight",
    "large_size_1":                "large_size_1",
    "large_cutoff_freedom":        "large_cutoff_freedom",
    "tradeoff":                    "tradeoff",
    "flpens":                      "flpens",
    "semester" : 'semester'
}

reverse_metrics_map = {v: k for k, v in metrics_map.items()}

col_map = {
    "Exam Group": "exam_id",
    "Exam Block": "block",
    # if your sheets ever include a “Semester” column, you could map it too:
    # "Semester":    "semester"
}
inv_map = {v: k for k, v in col_map.items()}
inv_map["semester"] = "Semester"


# map your CSV’s columns → the DB columns
col_map_schedule = {
    "Exam Group":   "exam_id",
    "slot":      "slot",
    "Semester":  "semester",
    # if your CSV also has a “Faculty” column, you could add:
    # "Faculty":   "faculty",
}
inv_map_schedule = {v: k for k, v in col_map_schedule.items()}

import os
import glob
import pandas as pd
from sqlalchemy import create_engine, text
from connection import DatabaseConnection

# ── CONFIGURE ────────────────────────────────────────────────────────────────
DB_URL        = "sqlite:///schedules.db"
#"sqlite:///schedules.db"
METRICS_DIR   = SAVE_PATH + "/metrics/"    # ← change me
SCHEDULE_DIR  = SAVE_PATH + "/schedules/"  # ← change me
SEMESTER= "sp25"
# ───────────────────────────────────────────────────────────────────────────────

engine = DatabaseConnection.get_engine()

# 1) Create tables if they don't exist
with engine.begin() as conn:
    conn.exec_driver_sql("""
    CREATE TABLE IF NOT EXISTS schedules (
      schedule_id   TEXT    PRIMARY KEY,
      display_name  TEXT,
      max_slot      INTEGER
    );
    """)
    conn.exec_driver_sql("""
    CREATE TABLE IF NOT EXISTS metrics (
      schedule_id             TEXT    PRIMARY KEY
                                 REFERENCES schedules(schedule_id),
      conflicts               INTEGER,
      quints                  INTEGER,
      quads                   INTEGER,
      four_in_five            INTEGER,
      triple_in_24h           INTEGER,
      triple_in_same_day      INTEGER,
      three_in_four           INTEGER,
      evening_morning_b2b     INTEGER,
      other_b2b               INTEGER,
      two_in_three            INTEGER,
      singular_late           INTEGER,
      two_large_gap           INTEGER,
      avg_max                 FLOAT,
      lateness                INTEGER,
      size_cutoff             INTEGER,
      reserved                INTEGER,
      num_blocks              INTEGER,
      alpha                   FLOAT,
      gamma                   FLOAT,
      delta                   FLOAT,
      vega                    FLOAT,
      theta                   FLOAT,
      large_block_size        FLOAT,
      large_exam_weight       FLOAT,
      large_block_weight      FLOAT,
      large_size_1            FLOAT,
      large_cutoff_freedom    FLOAT,
      tradeoff                FLOAT,
      flpens                  FLOAT,
      semester                TEXT
    );
    """)
    conn.exec_driver_sql("""
    CREATE TABLE IF NOT EXISTS slots (
      schedule_id   TEXT    REFERENCES schedules(schedule_id),
      slot_number   INTEGER,
      present       INTEGER,
      PRIMARY KEY (schedule_id, slot_number)
    );
    """)
    conn.exec_driver_sql("""
    CREATE TABLE IF NOT EXISTS schedule_details (
      schedule_id   TEXT    NOT NULL
                                 REFERENCES schedules(schedule_id),
      exam_id       TEXT    NOT NULL,
      slot          INTEGER,
      semester      TEXT,
      PRIMARY KEY (schedule_id, exam_id)
    );
    """)
    conn.exec_driver_sql("""
    CREATE TABLE IF NOT EXISTS schedule_plots (
      schedule_id   TEXT    NOT NULL
                                 REFERENCES schedules(schedule_id),
      sched_plot       TEXT    NOT NULL,
      last_plot          INTEGER,
      semester      TEXT,
      PRIMARY KEY (schedule_id)
    );
    """)
    conn.exec_driver_sql("""
    CREATE TABLE IF NOT EXISTS pinned_schedules (
      id            INTEGER PRIMARY KEY AUTOINCREMENT,
      user_id       TEXT    NOT NULL,
      schedule_id   TEXT    NOT NULL
                                 REFERENCES schedules(schedule_id),
      name          TEXT,
      data          TEXT,    -- JSON data stored as text
      created       TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
      UNIQUE(user_id, schedule_id)  -- Prevent duplicate pins
    );
    """)

    # FIXED: Corrected slider_recordings table
    conn.exec_driver_sql("""
        CREATE TABLE IF NOT EXISTS slider_recordings (
          id            INTEGER PRIMARY KEY AUTOINCREMENT,
          session_id    TEXT    NOT NULL,
          slider_key    TEXT    NOT NULL,
          value         FLOAT,
          min_value     FLOAT,
          max_value     FLOAT,    -- ← FIXED: was "FLIAT"
          created       TIMESTAMP DEFAULT CURRENT_TIMESTAMP,  -- ← FIXED: added "TIMESTAMP"
          UNIQUE(session_id, slider_key)
        );
    """)
    
    # NEW: Add slider_configs table
    conn.exec_driver_sql("""
        CREATE TABLE IF NOT EXISTS slider_configs (
          id            INTEGER NOT NULL PRIMARY KEY,
          name          VARCHAR(100) NOT NULL,
          description   TEXT,
          thresholds    JSON NOT NULL,
          timestamp     DATETIME,
          UNIQUE (name)
        );
    """)

    conn.exec_driver_sql("""
        CREATE TABLE IF NOT EXISTS block_assignments (
          block_id  TEXT    NOT NULL,
          exam_id   TEXT    NOT NULL,
          block     INTEGER,
          semester  TEXT,
          PRIMARY KEY (block_id, exam_id)
        );
        """)

OperationalError: (sqlite3.OperationalError) disk I/O error
[SQL: 
    CREATE TABLE IF NOT EXISTS schedules (
      schedule_id   TEXT    PRIMARY KEY,
      display_name  TEXT,
      max_slot      INTEGER
    );
    ]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [98]:
import pandas as pd
import os 
#from backend.config import SAVE_PATH
SAVE_PATH = os.getenv('SAVE_PATH', '/Users/adamshafikjovine/Documents/BOScheduling/results/sp25')  # Default to 'data' if SAVE_PATH is not set
print('SAVE PATH:', SAVE_PATH)
import glob 
date_prefix =  "*"

metrics_dir = os.path.join(SAVE_PATH, 'blocks')
pattern = os.path.join(metrics_dir, f"{date_prefix}*.csv")
for f in [os.path.basename(p) for p in glob.glob(pattern)]:
  schedule = pd.read_csv(SAVE_PATH + '/blocks/' + f)
  schedule = schedule.loc[:, ~schedule.columns.str.contains('Unnamed')]
  schedule.to_csv(SAVE_PATH + '/blocks/' + f)


SAVE PATH: /Users/adamshafikjovine/Documents/BOScheduling/results/sp25


ImportError: attempted relative import with no known parent package

In [2]:
from sqlalchemy import create_engine, text
import os

# 1) Point this at the same file your Flask app is using:
DB_PATH = os.path.expanduser(
    '/Users/adamshafikjovine/Documents/BOScheduling/backend/app/schedules.db'
)
if not os.path.exists(DB_PATH):
    raise FileNotFoundError(f"No DB found at {DB_PATH}")

# 2) Create an engine with a bit more timeout
engine = create_engine(
    f"sqlite:///{DB_PATH}",
    connect_args={"timeout": 30}    # wait up to 30s for any lock to clear
)

# 3) Run the CREATE TABLE DDL in a transaction
with engine.begin() as conn:
    conn.execute(text("""
    CREATE TABLE IF NOT EXISTS pinned_schedules (
      id            INTEGER PRIMARY KEY AUTOINCREMENT,
      user_id       TEXT    NOT NULL,
      schedule_id   TEXT    NOT NULL REFERENCES schedules(schedule_id),
      name          TEXT,
      data          TEXT,
      created       TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
      UNIQUE(user_id, schedule_id)
    )
    """))

engine.dispose()
print("✅ pinned_schedules table is now present in schedules.db")



✅ pinned_schedules table is now present in schedules.db


In [3]:
# simple_import_sliders.py
# Put your slider CSV files in a folder and adjust SLIDERS_DIR below

import os
import glob
import csv
from sqlalchemy import create_engine, text

# ADJUST THESE PATHS
SLIDERS_DIR = SAVE_PATH + "/sliders/"  # Change this to where your slider CSV files are
CHUNK_SIZE = 100

# Create engine directly
engine = create_engine("sqlite:///schedules.db", echo=False)

# Create the table
create_table_sql = text("""
CREATE TABLE IF NOT EXISTS slider_recordings (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT NOT NULL,
    slider_key TEXT NOT NULL,
    value FLOAT,
    min_value FLOAT,
    max_value FLOAT,
    created TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UNIQUE(session_id, slider_key)
)
""")

stmt = text("""
INSERT OR REPLACE INTO slider_recordings
    (session_id, slider_key, value, min_value, max_value, created)
VALUES
    (:session_id, :slider_key, :value, :min_value, :max_value, :created)
""")

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

print("🚀 Starting slider CSV import...")

with engine.begin() as conn:
    # Create table
    conn.execute(create_table_sql)
    print("✅ Table created/verified")
    
    total_inserted = 0
    
    for path in glob.glob(os.path.join(SLIDERS_DIR, "*.csv")):
        print(f"📁 Processing: {os.path.basename(path)}")
        
        with open(path, newline="") as f:
            reader = csv.DictReader(f)
            
            # Find columns (adjust these based on your CSV headers)
            session_col = next((c for c in reader.fieldnames if "session" in c.lower()), None)
            key_col = next((c for c in reader.fieldnames if "key" in c.lower() or "slider" in c.lower()), None)
            value_col = next((c for c in reader.fieldnames if "value" in c.lower()), None)
            min_col = next((c for c in reader.fieldnames if "min" in c.lower()), None)
            max_col = next((c for c in reader.fieldnames if "max" in c.lower()), None)
            created_col = next((c for c in reader.fieldnames if "created" in c.lower() or "timestamp" in c.lower()), None)
            
            if not session_col or not key_col or not value_col:
                print(f"   ⚠️ Skipping {path}: missing required columns")
                print(f"      Available columns: {reader.fieldnames}")
                continue
            
            # Build batch for this file
            batch = []
            for row in reader:
                session_id = row[session_col].strip() if row[session_col] else None
                slider_key = row[key_col].strip() if row[key_col] else None
                
                if not session_id or not slider_key:
                    continue
                
                try:
                    value = float(row[value_col]) if row[value_col] else None
                    min_value = float(row[min_col]) if min_col and row[min_col] else None
                    max_value = float(row[max_col]) if max_col and row[max_col] else None
                    created = row[created_col] if created_col and row[created_col] else None
                    
                    batch.append({
                        "session_id": session_id,
                        "slider_key": slider_key,
                        "value": value,
                        "min_value": min_value,
                        "max_value": max_value,
                        "created": created
                    })
                except (ValueError, TypeError) as e:
                    print(f"⚠️ Skipping invalid row: {e}")
                    continue

            print(f"   📊 Total records to insert: {len(batch)}")
            
            # Insert in chunks
            if batch:
                chunk_count = 0
                for chunk in chunks(batch, CHUNK_SIZE):
                    try:
                        conn.execute(stmt, chunk)
                        chunk_count += 1
                        total_inserted += len(chunk)
                        print(f"   ✅ Inserted chunk {chunk_count} ({len(chunk)} records)")
                    except Exception as e:
                        print(f"   ❌ Error inserting chunk {chunk_count}: {e}")
                        continue
                
                print(f"   🎉 Completed {os.path.basename(path)}: {len(batch)} records processed")
            else:
                print(f"   ⚠️ No valid records found in {os.path.basename(path)}")

print(f"\n🎉 Total records inserted: {total_inserted}")

# Verify
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM slider_recordings"))
        count = result.scalar()
        print(f"✅ Verification: {count} records now in slider_recordings")
except Exception as e:
    print(f"❌ Verification failed: {e}")

print("🎉 Done!")


NameError: name 'SAVE_PATH' is not defined

In [1]:
#!/usr/bin/env python3
import sqlite3
import time
from sqlite3 import OperationalError

DB_PATH = '/Users/adamshafikjovine/Documents/BOScheduling/backend/app/schedules.db'

def get_connection(path, retries=5, backoff=0.1):
    for i in range(retries):
        try:
            # give SQLite a longer timeout for locks
            return sqlite3.connect(path, timeout=30.0)
        except OperationalError as e:
            if 'locked' in str(e).lower() and i < retries - 1:
                time.sleep(backoff * (2 ** i))
            else:
                raise
    raise OperationalError("Could not open the database (locked).")

def ensure_tables(conn):
    cur = conn.cursor()
    # slider_recordings with slider_key instead of name
    cur.execute("""
        CREATE TABLE IF NOT EXISTS slider_recordings (
          id            INTEGER PRIMARY KEY AUTOINCREMENT,
          session_id    TEXT    NOT NULL,
          slider_key    TEXT    NOT NULL,
          value         REAL,
          min_value     REAL,
          max_value     REAL,
          created       TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
          UNIQUE(session_id, slider_key)
        );
    """)
    # slider_configs
    cur.execute("""
        CREATE TABLE IF NOT EXISTS slider_configs (
          id            INTEGER NOT NULL PRIMARY KEY,
          name          VARCHAR(100) NOT NULL UNIQUE,
          description   TEXT,
          thresholds    JSON NOT NULL,
          timestamp     DATETIME
        );
    """)
    conn.commit()

def seed_slider_recordings(conn, recordings):
    cur = conn.cursor()
    cur.executemany("""
        INSERT INTO slider_recordings
            (session_id, slider_key, value, min_value, max_value, created)
        VALUES
            (?,          ?,          ?,     ?,         ?,         ?)
    """, [
        (
            r['session_id'],
            r['slider_key'],
            r['value'],
            r['min_value'],
            r['max_value'],
            r['created']
        ) for r in recordings
    ])
    conn.commit()

if __name__ == "__main__":
    conn = get_connection(DB_PATH)
    ensure_tables(conn)

    recordings = [
        {
            'session_id': 'fc54723b-d4df-49b1-8908-3fd366a27619',
            'slider_key': 'avg_max',
            'value':      17.2093577981651364,
            'min_value':  14.1470336391437303,
            'max_value':  17.2093577981651364,
            'created':    '2025-06-28 22:17:02.790116'
        },
        {
            'session_id': 'fc54723b-d4df-49b1-8908-3fd366a27619',
            'slider_key': 'back_to_back',
            'value':      3096.0,
            'min_value':  1122.0,
            'max_value':  3096.0,
            'created':    '2025-06-28 22:17:02.790122'
        },
        {
            'session_id': 'fc54723b-d4df-49b1-8908-3fd366a27619',
            'slider_key': 'conflicts',
            'value':      14.0,
            'min_value':  0.0,
            'max_value':  14.0,
            'created':    '2025-06-28 22:17:02.790123'
        },
    ]

    seed_slider_recordings(conn, recordings)
    conn.close()
    print("✅ slider_recordings seeded successfully")




✅ slider_recordings seeded successfully


In [2]:
import pandas as pd
from sqlalchemy import create_engine

# Adjust if needed
#DB_URL = "sqlite:///schedules.db"

# 1) Create the engine
engine = DatabaseConnection.get_engine()
# 2) Read the table into a DataFrame
df_blocks = pd.read_sql_query("SELECT * FROM block_assignments", engine)

# 3) Print it out
print(df_blocks.to_string(index=False))


NameError: name 'DatabaseConnection' is not defined

OperationalError: database is locked

In [106]:
!df -h .

Filesystem      Size    Used   Avail Capacity iused ifree %iused  Mounted on
/dev/disk3s5   460Gi   364Gi    61Gi    86%    3.5M  636M    1%   /System/Volumes/Data


In [127]:
import os
import glob
import csv
from sqlalchemy import create_engine, text

# ── CONFIG ────────────────────────────────────────────────────────────────────
SAVE_PATH = "."  # Adjust this to your actual path
SCHEDULE_DIR = '/Users/adamshafikjovine/Documents/BOScheduling/results/sp25/schedules'#os.path.join(SAVE_PATH, "schedules")  # adjust if needed
SEMESTER = "sp25"
# ───────────────────────────────────────────────────────────────────────────────

print(f"🚀 Processing schedules from: {SCHEDULE_DIR}")

# 1) Create engine directly - no bullshit
engine = create_engine("sqlite:///schedules.db", echo=False)

# 2) Prepare statements
delete_sql = text("DELETE FROM schedule_details WHERE schedule_id = :schedule_id")
insert_sql = text("""
INSERT OR REPLACE INTO schedule_details
  (schedule_id, exam_id, slot, faculty, semester)
VALUES
  (:schedule_id, :exam_id, :slot, :faculty, :semester)
""")

# 3) Create table if it doesn't exist
create_table_sql = text("""
CREATE TABLE IF NOT EXISTS schedule_details (
  schedule_id   TEXT    NOT NULL,
  exam_id       TEXT    NOT NULL,
  slot          INTEGER,
  faculty       TEXT,
  semester      TEXT,
  PRIMARY KEY (schedule_id, exam_id)
)
""")

# 4) Process files
with engine.begin() as conn:
    # Create table
    conn.execute(create_table_sql)
    print("✅ Table created/verified")
    
    total_processed = 0
    
    # For each CSV, read only the two needed columns and batch-insert
    for path in glob.glob(os.path.join(SCHEDULE_DIR, "*.csv")):
        print(f"📁 Processing: {os.path.basename(path)}")
        schedule_id = os.path.splitext(os.path.basename(path))[0]

        # Remove old rows for this schedule
        conn.execute(delete_sql, {"schedule_id": schedule_id})

        # Stream the CSV, pulling out only Exam Group & slot
        batch = []
        with open(path, newline="") as f:
            reader = csv.DictReader(f)
            
            # Find the columns automatically
            exam_col = None
            slot_col = None
            
            for col in reader.fieldnames:
                if "exam" in col.lower() and "group" in col.lower():
                    exam_col = col
                elif "slot" in col.lower():
                    slot_col = col
            
            if not exam_col or not slot_col:
                print(f"   ⚠️ Skipping {path}: couldn't find exam group or slot columns")
                print(f"      Available columns: {reader.fieldnames}")
                continue

            for row in reader:
                eid = row.get(exam_col, "").strip()
                if not eid:
                    continue
                try:
                    sl = int(row.get(slot_col, 0))
                except ValueError:
                    continue
                batch.append({
                    "schedule_id": schedule_id,
                    "exam_id": eid,
                    "slot": sl,
                    "faculty": "",
                    "semester": SEMESTER
                })

        # Bulk insert all rows for this schedule in one go
        if batch:
            try:
                conn.execute(insert_sql, batch)
                total_processed += len(batch)
                print(f"   ✅ Inserted {len(batch)} records for {schedule_id}")
            except Exception as e:
                print(f"   ❌ Error inserting {schedule_id}: {e}")
        else:
            print(f"   ⚠️ No valid records found in {schedule_id}")

print(f"\n🎉 Total records processed: {total_processed}")

# Verify
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM schedule_details WHERE semester = :semester"), 
                            {"semester": SEMESTER})
        count = result.scalar()
        print(f"✅ Verification: {count} records now in schedule_details for {SEMESTER}")
except Exception as e:
    print(f"❌ Verification failed: {e}")

print("🎉 Done!")


🚀 Processing schedules from: /Users/adamshafikjovine/Documents/BOScheduling/results/sp25/schedules
✅ Table created/verified
📁 Processing: 20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e.csv
   ✅ Inserted 566 records for 20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e
📁 Processing: 20250618_1126422324i151-dde79de178361831f7a1ac9de0d67af5.csv
   ✅ Inserted 566 records for 20250618_1126422324i151-dde79de178361831f7a1ac9de0d67af5
📁 Processing: 20250624_183312i26par___-951b9a8c20bdd40024d8b6601e1e26f3.csv
   ✅ Inserted 566 records for 20250624_183312i26par___-951b9a8c20bdd40024d8b6601e1e26f3
📁 Processing: 20250619_072545i166fl_test-0bbb84af26d32dd100ccbc7438fce590.csv
   ✅ Inserted 566 records for 20250619_072545i166fl_test-0bbb84af26d32dd100ccbc7438fce590
📁 Processing: INITIAL20250624_185106i286thomp___-44f0bb5c487a180b8635e2d53d28a9e9.csv
   ✅ Inserted 566 records for INITIAL20250624_185106i286thomp___-44f0bb5c487a180b8635e2d53d28a9e9
📁 Processing: INITIAL20250624_18

In [129]:
import pandas as pd
from sqlalchemy import create_engine

#DB_URL = "sqlite:///schedules.db"
#engine = create_engine(DB_URL, echo=False)

# Read only the first 5 rows via SQL LIMIT
df_head = pd.read_sql_query(
    "SELECT schedule_id, exam_id, slot, faculty, semester FROM schedule_details LIMIT 5",
    engine
)

print(df_head.to_string(index=False))



                                               schedule_id exam_id  slot faculty semester
20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e  1-9379     8             sp25
20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e   MLG32     3             sp25
20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e   MLG40     3             sp25
20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e  1-9227     7             sp25
20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e 1-19286    10             sp25


In [130]:
import os
import glob
import pandas as pd
from sqlalchemy import create_engine, text

# ── CONFIG ────────────────────────────────────────────────────────────────────
SAVE_PATH = "."  # Adjust this to your actual path
METRICS_DIR = '/Users/adamshafikjovine/Documents/BOScheduling/results/sp25/metrics'#os.path.join(SAVE_PATH, "metrics")  # adjust if needed
SEMESTER = "sp25"
# ───────────────────────────────────────────────────────────────────────────────

# Metrics mapping (add this if you don't have it)
metrics_map = {
    "conflicts": "conflicts",
    "quints": "quints",
    "quads": "quads",
    "four in five slots": "four_in_five",
    "triple in 24h (no gaps)": "triple_in_24h",
    "triple in same day (no gaps)": "triple_in_same_day",
    "three in four slots": "three_in_four",
    "evening/morning b2b": "evening_morning_b2b",
    "other b2b": "other_b2b",
    "two in three slots": "two_in_three",
    "singular late exam": "singular_late",
    "two exams, large gap": "two_large_gap",
    "avg_max": "avg_max",
    "lateness": "lateness",
    "size_cutoff": "size_cutoff",
    "reserved": "reserved",
    "num_blocks": "num_blocks",
    "alpha": "alpha",
    "gamma": "gamma",
    "delta": "delta",
    "vega": "vega",
    "theta": "theta",
    "large_block_size": "large_block_size",
    "large_exam_weight": "large_exam_weight",
    "large_block_weight": "large_block_weight",
    "large_size_1": "large_size_1",
    "large_cutoff_freedom": "large_cutoff_freedom",
    "tradeoff": "tradeoff",
    "flpens": "flpens",
    "semester": "semester"
}

print(f"🚀 Processing metrics from: {METRICS_DIR}")

# Create engine directly
engine = create_engine("sqlite:///schedules.db", echo=False)

# Create table if it doesn't exist
create_table_sql = text("""
CREATE TABLE IF NOT EXISTS metrics (
  schedule_id             TEXT    PRIMARY KEY,
  conflicts               INTEGER,
  quints                  INTEGER,
  quads                   INTEGER,
  four_in_five            INTEGER,
  triple_in_24h           INTEGER,
  triple_in_same_day      INTEGER,
  three_in_four           INTEGER,
  evening_morning_b2b     INTEGER,
  other_b2b               INTEGER,
  two_in_three            INTEGER,
  singular_late           INTEGER,
  two_large_gap           INTEGER,
  avg_max                 FLOAT,
  lateness                INTEGER,
  size_cutoff             INTEGER,
  reserved                INTEGER,
  num_blocks              INTEGER,
  alpha                   FLOAT,
  gamma                   FLOAT,
  delta                   FLOAT,
  vega                    FLOAT,
  theta                   FLOAT,
  large_block_size        FLOAT,
  large_exam_weight       FLOAT,
  large_block_weight      FLOAT,
  large_size_1            FLOAT,
  large_cutoff_freedom    FLOAT,
  tradeoff                FLOAT,
  flpens                  FLOAT,
  semester                TEXT
)
""")

with engine.begin() as conn:
    # Create table
    conn.execute(create_table_sql)
    print("✅ Metrics table created/verified")
    
    total_processed = 0
    
    for path in glob.glob(os.path.join(METRICS_DIR, "*.csv")):
        print(f"📁 Processing: {os.path.basename(path)}")
        basename = os.path.splitext(os.path.basename(path))[0]
        
        try:
            df = pd.read_csv(path)
            if len(df) == 0:
                print(f"   ⚠️ Empty file: {basename}")
                continue
                
            row = df.iloc[0]  # first row only

            # build the dict we'll bind to the INSERT
            data = {"schedule_id": basename, "semester": SEMESTER}
            print(f"   📊 Processing schedule: {basename}")
            
            for csv_col, sql_col in metrics_map.items():
                if csv_col not in row or pd.isna(row[csv_col]):
                    # skip any missing columns or NaN values
                    continue
                    
                val = row[csv_col]
                
                # cast to the right type
                if sql_col in (
                    "conflicts","quints","quads","four_in_five",
                    "triple_in_24h","triple_in_same_day","three_in_four",
                    "evening_morning_b2b","other_b2b","two_in_three",
                    "singular_late","two_large_gap",
                    "lateness","size_cutoff","reserved","num_blocks"
                ):
                    try:
                        data[sql_col] = int(float(val))  # Convert to float first, then int
                    except (ValueError, TypeError):
                        print(f"   ⚠️ Skipping invalid integer value for {csv_col}: {val}")
                        continue
                else:
                    try:
                        data[sql_col] = float(val)
                    except (ValueError, TypeError):
                        print(f"   ⚠️ Skipping invalid float value for {csv_col}: {val}")
                        continue

            # Build the INSERT … VALUES (:col1,:col2,…)
            if len(data) > 2:  # More than just schedule_id and semester
                cols = ", ".join(data.keys())
                params = ", ".join(f":{c}" for c in data)
                stmt = text(f"INSERT OR REPLACE INTO metrics ({cols}) VALUES ({params})")
                conn.execute(stmt, data)
                total_processed += 1
                print(f"   ✅ Inserted metrics for {basename} ({len(data)-2} metrics)")
            else:
                print(f"   ⚠️ No valid metrics found for {basename}")
                
        except Exception as e:
            print(f"   ❌ Error processing {basename}: {e}")
            continue

print(f"\n🎉 Total files processed: {total_processed}")

# Verify
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM metrics WHERE semester = :semester"), 
                            {"semester": SEMESTER})
        count = result.scalar()
        print(f"✅ Verification: {count} records now in metrics for {SEMESTER}")
except Exception as e:
    print(f"❌ Verification failed: {e}")

print("🎉 Done!")


🚀 Processing metrics from: /Users/adamshafikjovine/Documents/BOScheduling/results/sp25/metrics
✅ Metrics table created/verified
📁 Processing: 20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e.csv
   📊 Processing schedule: 20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e
   ✅ Inserted metrics for 20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e (29 metrics)
📁 Processing: 20250618_1126422324i151-dde79de178361831f7a1ac9de0d67af5.csv
   📊 Processing schedule: 20250618_1126422324i151-dde79de178361831f7a1ac9de0d67af5
   ✅ Inserted metrics for 20250618_1126422324i151-dde79de178361831f7a1ac9de0d67af5 (29 metrics)
📁 Processing: 20250624_183312i26par___-951b9a8c20bdd40024d8b6601e1e26f3.csv
   📊 Processing schedule: 20250624_183312i26par___-951b9a8c20bdd40024d8b6601e1e26f3
   ✅ Inserted metrics for 20250624_183312i26par___-951b9a8c20bdd40024d8b6601e1e26f3 (29 metrics)
📁 Processing: 20250619_072545i166fl_test-0bbb84af26d32dd100ccbc7438fce590.csv
   📊 Processing schedul

In [133]:
import os
import glob
import csv
from collections import defaultdict
from sqlalchemy import create_engine, text

# ── CONFIG ────────────────────────────────────────────────────────────────────
SAVE_PATH = "."  # Adjust this to your actual path
SCHEDULE_DIR ='/Users/adamshafikjovine/Documents/BOScheduling/results/sp25/schedules'# os.path.join(SAVE_PATH, "schedules")  # adjust if needed
SEMESTER = "sp25"
# ───────────────────────────────────────────────────────────────────────────────

print(f"🚀 Processing slots from: {SCHEDULE_DIR}")

# Create engine directly
engine = create_engine("sqlite:///schedules.db", echo=False)

# Create tables if they don't exist
create_slots_table_sql = text("""
CREATE TABLE IF NOT EXISTS slots (
  schedule_id   TEXT    NOT NULL,
  slot_number   INTEGER NOT NULL,
  present       INTEGER DEFAULT 0,
  PRIMARY KEY (schedule_id, slot_number)
)
""")

create_schedules_table_sql = text("""
CREATE TABLE IF NOT EXISTS schedules (
  schedule_id   TEXT    PRIMARY KEY,
  display_name  TEXT,
  max_slot      INTEGER
)
""")

# Prepare statements
delete_slots_sql = text("DELETE FROM slots WHERE schedule_id = :schedule_id")
insert_slots_sql = text("""
INSERT OR REPLACE INTO slots
  (schedule_id, slot_number, present)
VALUES
  (:schedule_id, :slot_number, :present)
""")

with engine.begin() as conn:
    # Create tables
    conn.execute(create_slots_table_sql)
    conn.execute(create_schedules_table_sql)
    print("✅ Tables created/verified")
    
    total_schedules = 0
    total_slots = 0
    
    # For each CSV, find unique slots and populate the slots table
    for path in glob.glob(os.path.join(SCHEDULE_DIR, "*.csv")):
        schedule_id = os.path.splitext(os.path.basename(path))[0]
        
        print(f"📁 Processing schedule: {schedule_id}")
        
        # Remove old slot entries for this schedule
        conn.execute(delete_slots_sql, {"schedule_id": schedule_id})
        
        # Collect unique slots from this CSV
        unique_slots = set()
        
        with open(path, newline="") as f:
            reader = csv.DictReader(f)
            
            # Find slot column automatically
            slot_col = None
            for col in reader.fieldnames:
                if "slot" in col.lower():
                    slot_col = col
                    break
            
            if not slot_col:
                print(f"   ⚠️ Skipping {schedule_id}: no slot column found")
                print(f"      Available columns: {reader.fieldnames}")
                continue
            
            for row in reader:
                try:
                    slot = int(row.get(slot_col, 0))
                    if slot > 0:  # assuming slot 0 means "no slot assigned"
                        unique_slots.add(slot)
                except (ValueError, TypeError):
                    continue
        
        # Prepare batch insert for slots
        slots_batch = []
        for slot_num in sorted(unique_slots):
            # present = 1 means this slot is used in this schedule
            slots_batch.append({
                "schedule_id": schedule_id,
                "slot_number": slot_num,
                "present": 1
            })
        
        # Insert all slots for this schedule
        if slots_batch:
            try:
                conn.execute(insert_slots_sql, slots_batch)
                total_schedules += 1
                total_slots += len(slots_batch)
                print(f"   ✅ Inserted {len(slots_batch)} slots for {schedule_id}")
                
                # Also insert/update the schedule record
                schedule_insert_sql = text("""
                INSERT OR REPLACE INTO schedules (schedule_id, display_name, max_slot)
                VALUES (:schedule_id, :display_name, :max_slot)
                """)
                
                max_slot = max(slot_num for slot_num in unique_slots)
                conn.execute(schedule_insert_sql, {
                    "schedule_id": schedule_id,
                    "display_name": schedule_id,
                    "max_slot": max_slot
                })
                
            except Exception as e:
                print(f"   ❌ Error inserting slots for {schedule_id}: {e}")
        else:
            print(f"   ⚠️ No valid slots found for {schedule_id}")

    # 4) Update the schedules table with max_slot for each schedule
    print("🔄 Updating max_slot in schedules table...")
    update_max_slot_sql = text("""
    UPDATE schedules 
    SET max_slot = (
        SELECT MAX(slot_number) 
        FROM slots 
        WHERE slots.schedule_id = schedules.schedule_id
    )
    WHERE schedule_id IN (
        SELECT DISTINCT schedule_id FROM slots
    )
    """)
    
    try:
        conn.execute(update_max_slot_sql)
        print("   ✅ Updated max_slot values")
    except Exception as e:
        print(f"   ⚠️ Error updating max_slot: {e}")

print(f"\n🎉 Processing complete!")
print(f"📊 Total schedules processed: {total_schedules}")
print(f"📊 Total slots inserted: {total_slots}")

# Verify
try:
    with engine.connect() as conn:
        # Check slots
        result = conn.execute(text("SELECT COUNT(*) FROM slots"))
        slots_count = result.scalar()
        
        # Check schedules
        result = conn.execute(text("SELECT COUNT(*) FROM schedules WHERE max_slot IS NOT NULL"))
        schedules_count = result.scalar()
        
        print(f"✅ Verification: {slots_count} total slots, {schedules_count} schedules with max_slot")
        
        # Show some sample data
        result = conn.execute(text("SELECT schedule_id, max_slot FROM schedules WHERE max_slot IS NOT NULL LIMIT 3"))
        samples = result.fetchall()
        if samples:
            print("📋 Sample schedules:")
            for row in samples:
                print(f"   {row.schedule_id}: max_slot = {row.max_slot}")
                
except Exception as e:
    print(f"❌ Verification failed: {e}")

print("🎉 Slots table population complete!")

🚀 Processing slots from: /Users/adamshafikjovine/Documents/BOScheduling/results/sp25/schedules
✅ Tables created/verified
📁 Processing schedule: 20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e
   ✅ Inserted 23 slots for 20250626_094553i130thomp:-46593c69bfa61c77f122951667ba2f9e
📁 Processing schedule: 20250618_1126422324i151-dde79de178361831f7a1ac9de0d67af5
   ✅ Inserted 20 slots for 20250618_1126422324i151-dde79de178361831f7a1ac9de0d67af5
📁 Processing schedule: 20250624_183312i26par___-951b9a8c20bdd40024d8b6601e1e26f3
   ✅ Inserted 21 slots for 20250624_183312i26par___-951b9a8c20bdd40024d8b6601e1e26f3
📁 Processing schedule: 20250619_072545i166fl_test-0bbb84af26d32dd100ccbc7438fce590
   ✅ Inserted 20 slots for 20250619_072545i166fl_test-0bbb84af26d32dd100ccbc7438fce590
📁 Processing schedule: INITIAL20250624_185106i286thomp___-44f0bb5c487a180b8635e2d53d28a9e9
   ✅ Inserted 21 slots for INITIAL20250624_185106i286thomp___-44f0bb5c487a180b8635e2d53d28a9e9
📁 Processing schedule: IN

In [77]:

def verify_slots_population():
    """Verify that the slots table was populated correctly"""
    
    conn = sqlite3.connect(DB_PATH)
    
    print("=== VERIFICATION: Slots Table Population ===\n")
    
    # 1) Check overall stats
    print("1. OVERALL STATISTICS:")
    print("-" * 40)
    
    # Count schedules and slots
    cur = conn.cursor()
    cur.execute("SELECT COUNT(DISTINCT schedule_id) FROM slots")
    num_schedules = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(*) FROM slots") 
    total_slots = cur.fetchone()[0]
    
    cur.execute("SELECT schedule_id, COUNT(*) as slot_count FROM slots GROUP BY schedule_id LIMIT 5")
    sample_counts = cur.fetchall()
    
    print(f"Total schedules with slots: {num_schedules}")
    print(f"Total slot records: {total_slots}")
    print(f"Average slots per schedule: {total_slots/num_schedules:.1f}")
    print("\nSample slot counts by schedule:")
    for sched_id, count in sample_counts:
        print(f"  {sched_id}: {count} slots")
    
    # 2) Check max_slot updates
    print(f"\n2. MAX_SLOT VERIFICATION:")
    print("-" * 40)
    
    cur.execute("""
    SELECT s.schedule_id, s.max_slot, MAX(sl.slot_number) as actual_max
    FROM schedules s
    LEFT JOIN slots sl ON s.schedule_id = sl.schedule_id
    WHERE s.schedule_id IN (SELECT DISTINCT schedule_id FROM slots)
    GROUP BY s.schedule_id
    LIMIT 5
    """)
    
    max_slot_check = cur.fetchall()
    print("Schedule ID | max_slot (table) | actual_max (calculated) | Match?")
    print("-" * 65)
    for sched_id, table_max, actual_max in max_slot_check:
        match = "✓" if table_max == actual_max else "✗"
        print(f"{sched_id[:15]:<15} | {table_max:<15} | {actual_max:<22} | {match}")
    
    # 3) Cross-verify with actual CSV files
    print(f"\n3. CSV FILE CROSS-VERIFICATION:")
    print("-" * 40)
    
    # Get first few schedule files to verify
    csv_files = glob.glob(os.path.join(SCHEDULE_DIR, "*.csv"))[:3]
    
    for csv_path in csv_files:
        schedule_id = os.path.splitext(os.path.basename(csv_path))[0]
        print(f"\nVerifying: {schedule_id}")
        
        # Get slots from database
        cur.execute("""
        SELECT slot_number FROM slots 
        WHERE schedule_id = ? 
        ORDER BY slot_number
        """, (schedule_id,))
        db_slots = set(row[0] for row in cur.fetchall())
        
        # Get slots from CSV file
        csv_slots = set()
        try:
            with open(csv_path, newline="") as f:
                reader = csv.DictReader(f)
                slot_col = "slot"
                
                for row in reader:
                    try:
                        slot = int(row.get(slot_col, 0))
                        if slot > 0:
                            csv_slots.add(slot)
                    except (ValueError, TypeError):
                        continue
        except Exception as e:
            print(f"  Error reading CSV: {e}")
            continue
        
        # Compare
        print(f"  CSV slots: {sorted(list(csv_slots))}")
        print(f"  DB slots:  {sorted(list(db_slots))}")
        print(f"  Match: {'✓' if csv_slots == db_slots else '✗'}")
        
        if csv_slots != db_slots:
            missing_in_db = csv_slots - db_slots
            extra_in_db = db_slots - csv_slots
            if missing_in_db:
                print(f"  Missing in DB: {sorted(list(missing_in_db))}")
            if extra_in_db:
                print(f"  Extra in DB: {sorted(list(extra_in_db))}")
    
    # 4) Show sample slot data
    print(f"\n4. SAMPLE SLOT RECORDS:")
    print("-" * 40)
    
    cur.execute("""
    SELECT schedule_id, slot_number, present 
    FROM slots 
    ORDER BY schedule_id, slot_number 
    LIMIT 20
    """)
    
    sample_slots = cur.fetchall()
    print("Schedule ID | Slot | Present")
    print("-" * 35)
    for sched_id, slot_num, present in sample_slots:
        print(f"{sched_id[:15]:<15} | {slot_num:<4} | {present}")
    
    # 5) Check for any issues
    print(f"\n5. POTENTIAL ISSUES CHECK:")
    print("-" * 40)
    
    # Check for schedules with no slots
    cur.execute("""
    SELECT s.schedule_id 
    FROM schedules s 
    LEFT JOIN slots sl ON s.schedule_id = sl.schedule_id 
    WHERE sl.schedule_id IS NULL
    """)
    no_slots = cur.fetchall()
    
    if no_slots:
        print(f"⚠️  Schedules with no slots: {len(no_slots)}")
        for (sched_id,) in no_slots[:5]:
            print(f"  - {sched_id}")
    else:
        print("✓ All schedules have slot records")
    
    # Check for slots with present != 1
    cur.execute("SELECT COUNT(*) FROM slots WHERE present != 1")
    non_present = cur.fetchone()[0]
    
    if non_present > 0:
        print(f"⚠️  Slots with present != 1: {non_present}")
    else:
        print("✓ All slots have present = 1")
    
    conn.close()
    print(f"\n{'='*50}")
    print("Verification complete!")

# Run the verification
verify_slots_population()

=== VERIFICATION: Slots Table Population ===

1. OVERALL STATISTICS:
----------------------------------------


OperationalError: no such table: slots

In [78]:
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///schedules.db")

query = text("""
SELECT *
  FROM metrics
 WHERE conflicts < 1
""")

with engine.connect() as conn:
    result = conn.execute(query)
    rows = result.fetchall()

    for row in rows:
        # row._mapping is a Mapping[str, Any]
        rec: dict = dict(row._mapping)
        print(rec)



OperationalError: (sqlite3.OperationalError) no such table: metrics
[SQL: 
SELECT *
  FROM metrics
 WHERE conflicts < 1
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
from typing import Any, Dict, List, Union
from sqlalchemy import create_engine, text

# Database URL (absolute or relative path)
DB_URL = "sqlite:///schedules.db"
engine = create_engine(DB_URL, echo=False)

# --- METRICS CRUD ---

from typing import Any, Dict, List, Union
from sqlalchemy import text

from typing import Any, Dict, List, Union
import pandas as pd
from sqlalchemy import text

def add_metric(
    data: Union[Dict[str, Any], pd.DataFrame],
    name: str
) -> None:
    """
    Insert or update one or more rows in metrics, tagging each with schedule_id=`name`.
    Accepts a single dict or a DataFrame whose columns are the *human* names.
    """
    # 1) normalize to list of human-readable dicts
    if isinstance(data, pd.DataFrame):
        human_rows = data.to_dict(orient="records")
    else:
        human_rows = [data]

    # 2) translate each into a DB-row dict, and inject schedule_id
    db_rows: List[Dict[str, Any]] = []
    for hr in human_rows:
        db_row = {"schedule_id": name}
        for human_col, val in hr.items():
            if human_col not in metrics_map:
                raise KeyError(f"No mapping for column {human_col!r}")
            db_col = metrics_map[human_col]
            db_row[db_col] = val
        db_rows.append(db_row)

    # 3) build the parameterized upsert based on the first row’s keys
    cols   = ", ".join(db_rows[0].keys())
    params = ", ".join(f":{c}" for c in db_rows[0].keys())
    stmt   = text(f"INSERT OR REPLACE INTO metrics ({cols}) VALUES ({params})")

    # 4) execute
    with engine.begin() as conn:
        for row in db_rows:
            conn.execute(stmt, row)

def get_metrics(
    conflicts_lt: Union[int, None] = None
) -> pd.DataFrame:
    """
    Fetch metrics as a DataFrame whose columns are the *human* names.
    If `conflicts_lt` is given, applies that filter on the DB column.
    """
    if conflicts_lt is None:
        sql    = text("SELECT * FROM metrics")
        params = {}
    else:
        sql    = text("SELECT * FROM metrics WHERE conflicts < :c")
        params = {"c": conflicts_lt}

    # read into DF with DB column names
    df = pd.read_sql_query(sql, engine, params=params)

    # rename to human column names
    df = df.rename(columns=reverse_metrics_map)

    return df
def delete_metric(schedule_id: str) -> None:
    """Remove the metrics row for a given schedule_id."""
    stmt = text("DELETE FROM metrics WHERE schedule_id = :sid")
    with engine.begin() as conn:
        conn.execute(stmt, {"sid": schedule_id})

# --- SCHEDULE CRUD ---



def add_schedule(schedule_id: str, df = None) -> None:
    """
    Read SAVE_PATH+'/schedules/{schedule_id}.csv' and upsert that
    schedule’s metadata, details and slots.
    - Deduplicates on exam_id (keeps last occurrence).
    - Uses INSERT OR REPLACE for schedule_details to avoid UNIQUE errors.
    """
    if df == None: 
        path = f"{SAVE_PATH}/schedules/{schedule_id}.csv"
        df = pd.read_csv(path)

    # rename CSV headers → DB columns
    df = df.rename(columns=col_map_schedule)

    # ensure required columns
    for col in ("exam_id", "slot"):
        if col not in df.columns:
            raise KeyError(f"Missing required column {col!r} in {path}")
    df["semester"] = df.get("semester", "")

    # dedupe on exam_id (keep last row for each exam)
    df = df.drop_duplicates(subset=["exam_id"], keep="last")

    # compute metadata
    display_name = schedule_id
    max_slot     = int(df["slot"].max())

    with engine.begin() as conn:
        # upsert schedules row
        conn.execute(
            text("""
                INSERT OR REPLACE INTO schedules
                    (schedule_id, display_name, max_slot)
                VALUES (:sid, :dn, :ms)
            """),
            {"sid": schedule_id, "dn": display_name, "ms": max_slot}
        )

        # clear old details & slots
        conn.execute(
            text("DELETE FROM schedule_details WHERE schedule_id = :sid"),
            {"sid": schedule_id}
        )
        conn.execute(
            text("DELETE FROM slots WHERE schedule_id = :sid"),
            {"sid": schedule_id}
        )

        # insert details with upsert
        used_slots = set()
        for _, row in df.iterrows():
            sid = schedule_id
            eid = str(row["exam_id"])
            sl  = int(row["slot"])
            sem = str(row["semester"])
            used_slots.add(sl)

            conn.execute(
                text("""
                    INSERT OR REPLACE INTO schedule_details
                        (schedule_id, exam_id, slot, semester)
                    VALUES (:sid, :eid, :sl, :sem)
                """),
                {"sid": sid, "eid": eid, "sl": sl, "sem": sem}
            )

        # insert slots
        for sl in used_slots:
            conn.execute(
                text("""
                    INSERT INTO slots
                        (schedule_id, slot_number, present)
                    VALUES (:sid, :sn, 1)
                """),
                {"sid": schedule_id, "sn": sl}
            )


def get_schedule(schedule_id: str) -> pd.DataFrame:
    """
    Read back schedule_details for `schedule_id` into a DataFrame
    with human‐friendly column names (“Exam ID”, “Slot”, “Semester”).
    """
    sql = text("""
        SELECT exam_id, slot, semester
          FROM schedule_details
         WHERE schedule_id = :sid
    """)
    df = pd.read_sql_query(sql, engine, params={"sid": schedule_id})

    # enforce types
    df = df.astype({"exam_id": str, "slot": int, "semester": str})

    # rename DB columns → CSV headers
    df = df.rename(columns=inv_map_schedule)

    return df


def delete_schedule(schedule_id: str) -> None:
    """Remove a schedule and all its dependent rows."""
    with engine.begin() as conn:
        conn.execute(text("DELETE FROM schedule_details WHERE schedule_id=:sid"),
                     {"sid": schedule_id})
        conn.execute(text("DELETE FROM slots WHERE schedule_id=:sid"),
                     {"sid": schedule_id})
        conn.execute(text("DELETE FROM schedules WHERE schedule_id=:sid"),
                     {"sid": schedule_id})

# --- BLOCK ASSIGNMENT CRUD ---
import pandas as pd
from sqlalchemy import text


def add_block_assignment(
    block_id: str,
    df = None 
) -> None:
    """
    Replace all block_assignments for `block_id` with the rows in `assignments`.
    `assignments` must have columns matching the keys of `col_map` (e.g. "Exam Group", "Exam Block"),
    plus optionally "Semester" if you add that to col_map.
    """

    if df == None : 
        assignments = pd.read_csv(SAVE_PATH + '/blocks/' + block_id + '.csv')
        df = assignments.rename(columns=col_map).copy()

    # 2. Ensure we have all required DB columns; if semester is missing, fill with empty string
    required = ["exam_id", "block", "semester"]
    if "semester" not in df.columns:
        df["semester"] = "sp25"

    # 3. Keep only the exact order the table expects
    df = df[required]

    with engine.begin() as conn:
        # delete existing for this block_id
        conn.execute(
            text("DELETE FROM block_assignments WHERE block_id = :bid"),
            {"bid": block_id}
        )
        # bulk‐insert via pandas
        df["block_id"] = block_id
        df = df[["block_id", "exam_id", "block", "semester"]]
        df.to_sql(
            "block_assignments",
            conn,
            if_exists="append",
            index=False,
            method="multi"
        )


def get_block_assignment(block_id: str) -> pd.DataFrame:
    """
    Fetch all block_assignments for `block_id` as a DataFrame with
    the original human‐readable column names (“Exam Group”, “Exam Block”,
    plus “Semester” if you mapped it).
    """
    sql = text("""
        SELECT block_id, exam_id, block, semester
          FROM block_assignments
         WHERE block_id = :bid
    """)
    df = pd.read_sql_query(sql, engine, params={"bid": block_id})

    # map back to your human labels
    # (and you can drop block_id or rename it if you like)
    out = df.rename(columns=inv_map)

    return out


In [79]:
import pandas as pd

def test_metrics():
    print("→ Testing metrics CRUD")
    # 1. Add or replace a metric
    m = pd.read_csv('/Users/adamshafikjovine/Documents/BOScheduling/results/sp25/metrics/20250614_075849i29-aad821a31c678b3ab8284e63c8145f73.csv')
    
    add_metric('20250620',m)
    # 2. Read back
    all_metrics = get_metrics()
    print("All metrics:", all_metrics)
    # 3. Filtered read
    print("Metrics with conflicts < 5:", get_metrics(conflicts_lt=5))
    # 4. Delete
    delete_metric("sch1")
    print("After delete:", get_metrics())

def test_schedule():
    print("\n→ Testing schedule CRUD")
    # Prepare some detail rows
    details = [
        {"exam_id": "e1", "slot": 1, "faculty": "Math"},
        {"exam_id": "e2", "slot": 2, "faculty": "CS"},
    ]
    # 1. Insert
    name = '20250615_103133i33-4905920bd771e14b7bf10cdf261ec65d'
    add_schedule(name)
    # 2. Fetch
    s = get_schedule(name)
    print("Schedule meta:", s)

    # 3. Delete
    delete_schedule("sch2")
    print("After delete:", get_schedule("sch2"))

def test_block_assignment():
    print("\n→ Testing block assignment CRUD")
    name = 'PAREGOBsize_cutoff243frontloading0num_blocks20'
    #assignments = pd.read_csv('/home/asj53/BOScheduling/results/sp25/blocks/'+ name+ '.csv')
    
    # 1. Insert/update
    add_block_assignment(name)
    # 2. Fetch
    df = get_block_assignment(name)
    print("Block assignments:\n", df)
    # (No delete provided—just overwrite next time if needed)

if __name__ == "__main__":
    #test_metrics()
    test_schedule()
    test_block_assignment()




→ Testing schedule CRUD


NameError: name 'add_schedule' is not defined

In [80]:
test_schedule()


→ Testing schedule CRUD


NameError: name 'add_schedule' is not defined

In [ ]:
import os
import glob
import pandas as pd
from sqlalchemy import create_engine, text
import re

# ── CONFIGURE ────────────────────────────────────────────────────────────────
DB_URL = "sqlite:///schedules.db"
PLOTS_DIR = "/home/asj53/BOScheduling/UI/pages/plots/"
SEMESTER = "sp25"
# ───────────────────────────────────────────────────────────────────────────────

engine = create_engine(DB_URL, echo=True)

def get_plot_files():
    """
    Scan the plots directory and return a dictionary mapping schedule_ids to their plot files
    """
    plot_data = {}
    
    # Get all PNG files in the plots directory
    png_files = glob.glob(os.path.join(PLOTS_DIR, "*.png"))
    
    for png_file in png_files:
        filename = os.path.basename(png_file)
        
        # Check if it's a distribution plot (last_plot)
        if filename.endswith("_distribution.png"):
            schedule_id = filename.replace("_distribution.png", "")
            if schedule_id not in plot_data:
                plot_data[schedule_id] = {"sched_plot": None, "last_plot": None}
            plot_data[schedule_id]["last_plot"] = filename
            
        # Check if it's a regular schedule plot (sched_plot)
        elif filename.endswith(".png") and not filename.endswith("_distribution.png"):
            schedule_id = filename.replace(".png", "")
            if schedule_id not in plot_data:
                plot_data[schedule_id] = {"sched_plot": None, "last_plot": None}
            plot_data[schedule_id]["sched_plot"] = filename
    
    return plot_data

def populate_schedule_details():
    """
    Populate the schedule_details table with plot information
    """
    plot_data = get_plot_files()
    
    print(f"Found {len(plot_data)} unique schedule IDs with plots")
    
    # First, let's see what we found
    for schedule_id, plots in plot_data.items():
        print(f"Schedule ID: {schedule_id}")
        print(f"  - sched_plot: {plots['sched_plot']}")
        print(f"  - last_plot: {plots['last_plot']}")
        print()
    
    with engine.begin() as conn:
        # Clear existing data for this semester (optional - remove if you want to keep existing data)
        conn.execute(text("DELETE FROM schedule_plots WHERE semester = :semester"), 
                    {"semester": SEMESTER})
        
        # Insert plot data
        for schedule_id, plots in plot_data.items():
            # Only insert if we have at least one plot
            if plots["sched_plot"] or plots["last_plot"]:
                conn.execute(text("""
                    INSERT OR REPLACE INTO schedule_plots
                    (schedule_id, sched_plot, last_plot, semester)
                    VALUES (:schedule_id, :sched_plot, :last_plot, :semester)
                """), {
                    "schedule_id": schedule_id,
                    "sched_plot": plots["sched_plot"],
                    "last_plot": 1 if plots["last_plot"] else 0,  # Assuming this is a boolean flag
                    "semester": SEMESTER
                })
        
        print(f"Successfully inserted/updated {len(plot_data)} records in schedule_plots table")

def verify_data():
    """
    Verify the data was inserted correctly
    """
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT schedule_id, sched_plot, last_plot, semester 
            FROM schedule_plots
            WHERE semester = :semester
            ORDER BY schedule_id
        """), {"semester": SEMESTER})
        
        print("\n" + "="*60)
        print("VERIFICATION: Data in schedule_plots table")
        print("="*60)
        
        for row in result:
            print(f"Schedule ID: {row.schedule_id}")
            print(f"  sched_plot: {row.sched_plot}")
            print(f"  last_plot: {row.last_plot}")
            print(f"  semester: {row.semester}")
            print()

def check_missing_schedules():
    """
    Check if there are schedule_ids in the schedules table that don't have plots
    """
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT s.schedule_id 
            FROM schedules s 
            LEFT JOIN schedule_plots sd ON s.schedule_id = sd.schedule_id 
                AND sd.semester = :semester
            WHERE sd.schedule_id IS NULL
        """), {"semester": SEMESTER})
        
        missing_schedules = [row.schedule_id for row in result]
        
        if missing_schedules:
            print(f"\nWARNING: Found {len(missing_schedules)} schedule IDs in schedules table without plots:")
            for schedule_id in missing_schedules[:10]:  # Show first 10
                print(f"  - {schedule_id}")
            if len(missing_schedules) > 10:
                print(f"  ... and {len(missing_schedules) - 10} more")
        else:
            print("\nAll schedules in the schedules table have corresponding plots!")

if __name__ == "__main__":
    print("Starting plot database population...")
    print(f"Scanning directory: {PLOTS_DIR}")
    print(f"Target semester: {SEMESTER}")
    print()
    
    # Check if plots directory exists
    if not os.path.exists(PLOTS_DIR):
        print(f"ERROR: Plots directory does not exist: {PLOTS_DIR}")
        exit(1)
    
    # Populate the database
    populate_schedule_details()
    
    # Verify the results
    verify_data()
    
    # Check for missing schedules
    check_missing_schedules()
    
    print("\nDone!")

Starting plot database population...
Scanning directory: /home/asj53/BOScheduling/UI/pages/plots/
Target semester: sp25

ERROR: Plots directory does not exist: /home/asj53/BOScheduling/UI/pages/plots/
Found 0 unique schedule IDs with plots
2025-07-02 14:23:01,184 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-02 14:23:01,193 INFO sqlalchemy.engine.Engine DELETE FROM schedule_plots WHERE semester = ?
2025-07-02 14:23:01,194 INFO sqlalchemy.engine.Engine [generated in 0.00219s] ('sp25',)
Successfully inserted/updated 0 records in schedule_plots table
2025-07-02 14:23:01,304 INFO sqlalchemy.engine.Engine COMMIT
2025-07-02 14:23:01,309 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-02 14:23:01,314 INFO sqlalchemy.engine.Engine 
            SELECT schedule_id, sched_plot, last_plot, semester 
            FROM schedule_plots
            WHERE semester = ?
            ORDER BY schedule_id
        
2025-07-02 14:23:01,317 INFO sqlalchemy.engine.Engine [generated in 0.00334s] ('

In [ ]:
# check_database.py - Simple script to check what's in your database

import sqlite3
import pandas as pd
from datetime import datetime

# Database path - update this to match your database location
DB_PATH = "schedules.db"

def check_database():
    """Simple function to check what's in the database"""
    
    print("🔍 CHECKING DATABASE")
    print("=" * 50)
    print(f"Database file: {DB_PATH}")
    print(f"Check time: {datetime.now()}")
    print()
    
    try:
        # Connect to database
        conn = sqlite3.connect(DB_PATH)
        
        # Check if database file exists and is accessible
        print("✅ Database connection successful")
        
        # Get list of all tables
        cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        print(f"📋 Found {len(tables)} tables:")
        for table in tables:
            print(f"   - {table[0]}")
        
        print()
        
        # Check slider_recordings table
        print("🎚️ SLIDER_RECORDINGS TABLE")
        print("-" * 30)
        
        try:
            # Get table info
            cursor = conn.execute("PRAGMA table_info(slider_recordings)")
            columns = cursor.fetchall()
            
            if columns:
                print("Columns:")
                for col in columns:
                    print(f"   - {col[1]} ({col[2]})")
                
                # Count total records
                cursor = conn.execute("SELECT COUNT(*) FROM slider_recordings")
                count = cursor.fetchone()[0]
                print(f"\n📊 Total records: {count}")
                
                if count > 0:
                    # Show first 5 records
                    print("\n📋 First 5 records:")
                    df = pd.read_sql_query("SELECT * FROM slider_recordings ORDER BY timestamp DESC LIMIT 5", conn)
                    print(df.to_string(index=False))
                    
                    # Show unique session IDs
                    cursor = conn.execute("SELECT DISTINCT session_id FROM slider_recordings")
                    sessions = cursor.fetchall()
                    print(f"\n🆔 Unique sessions: {len(sessions)}")
                    for session in sessions[:5]:  # Show first 5 sessions
                        print(f"   - {session[0]}")
                    
                    # Show unique slider keys
                    cursor = conn.execute("SELECT DISTINCT slider_key FROM slider_recordings")
                    sliders = cursor.fetchall()
                    print(f"\n🎛️ Unique sliders: {len(sliders)}")
                    for slider in sliders:
                        print(f"   - {slider[0]}")
                        
                else:
                    print("⚠️ No records found in slider_recordings table")
                    
            else:
                print("❌ slider_recordings table has no columns")
                
        except Exception as e:
            print(f"❌ Error checking slider_recordings: {e}")
        
        print()
        
        # Check slider_configs table
        print("⚙️ SLIDER_CONFIGS TABLE")
        print("-" * 25)
        
        try:
            # Get table info
            cursor = conn.execute("PRAGMA table_info(slider_configs)")
            columns = cursor.fetchall()
            
            if columns:
                print("Columns:")
                for col in columns:
                    print(f"   - {col[1]} ({col[2]})")
                
                # Count total records
                cursor = conn.execute("SELECT COUNT(*) FROM slider_configs")
                count = cursor.fetchone()[0]
                print(f"\n📊 Total configs: {count}")
                
                if count > 0:
                    # Show all configs
                    print("\n📋 All configurations:")
                    df = pd.read_sql_query("SELECT * FROM slider_configs ORDER BY timestamp DESC", conn)
                    print(df.to_string(index=False))
                else:
                    print("⚠️ No configurations saved yet")
                    
            else:
                print("❌ slider_configs table has no columns")
                
        except Exception as e:
            print(f"❌ Error checking slider_configs: {e}")
        
        # Close connection
        conn.close()
        
    except sqlite3.Error as e:
        print(f"❌ Database error: {e}")
    except FileNotFoundError:
        print(f"❌ Database file not found: {DB_PATH}")
        print("   Make sure you're running this from the correct directory")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

def quick_insert_test():
    """Quick test to insert a record and see if it works"""
    print("\n🧪 TESTING DATABASE WRITE")
    print("=" * 30)
    
    try:
        conn = sqlite3.connect(DB_PATH)
        
        # Try to insert a test record
        test_data = (
            'test-session-' + str(int(datetime.now().timestamp())),
            'test_slider',
            0.5,
            0.0,
            1.0,
            datetime.now().isoformat()
        )
        
        conn.execute("""
            INSERT INTO slider_recordings 
            (session_id, slider_key, value, min_value, max_value, timestamp)
            VALUES (?, ?, ?, ?, ?, ?)
        """, test_data)
        
        conn.commit()
        
        # Check if it was inserted
        cursor = conn.execute("SELECT COUNT(*) FROM slider_recordings WHERE session_id = ?", (test_data[0],))
        count = cursor.fetchone()[0]
        
        if count > 0:
            print("✅ Test record inserted successfully")
            print(f"   Session ID: {test_data[0]}")
        else:
            print("❌ Test record was not inserted")
        
        conn.close()
        
    except Exception as e:
        print(f"❌ Insert test failed: {e}")

def show_recent_activity():
    """Show recent activity in the database"""
    print("\n📅 RECENT ACTIVITY")
    print("=" * 20)
    
    try:
        conn = sqlite3.connect(DB_PATH)
        
        # Show records from last 24 hours
        query = """
        SELECT session_id, slider_key, value, timestamp 
        FROM slider_recordings 
        WHERE datetime(timestamp) > datetime('now', '-1 day')
        ORDER BY timestamp DESC 
        LIMIT 10
        """
        
        df = pd.read_sql_query(query, conn)
        
        if len(df) > 0:
            print(f"📊 {len(df)} records in last 24 hours:")
            print(df.to_string(index=False))
        else:
            print("⚠️ No activity in last 24 hours")
        
        conn.close()
        
    except Exception as e:
        print(f"❌ Error checking recent activity: {e}")

if __name__ == "__main__":
    print("🚀 SIMPLE DATABASE CHECKER")
    print("=" * 50)
    
    # Main check
    check_database()
    
    # Recent activity
    show_recent_activity()
    
    # Ask if user wants to run insert test
    print("\n" + "=" * 50)
    test_insert = input("🧪 Want to test inserting a record? (y/n): ").lower().strip()
    if test_insert == 'y':
        quick_insert_test()
        print("\nRe-checking database after test insert:")
        check_database()
    
    print("\n✨ Database check complete!")

# Alternative: Just run the main check function
# check_database()

🚀 SIMPLE DATABASE CHECKER
🔍 CHECKING DATABASE
Database file: schedules.db
Check time: 2025-07-02 14:23:01.826052

✅ Database connection successful
📋 Found 10 tables:
   - schedules
   - metrics
   - slots
   - schedule_details
   - schedule_plots
   - pinned_schedules
   - sqlite_sequence
   - slider_configs
   - slider_recordings
   - block_assignments

🎚️ SLIDER_RECORDINGS TABLE
------------------------------
Columns:
   - id (INTEGER)
   - session_id (TEXT)
   - slider_key (TEXT)
   - value (REAL)
   - min_value (REAL)
   - max_value (REAL)
   - timestamp (TIMESTAMP)

📊 Total records: 1

📋 First 5 records:
 id              session_id  slider_key  value  min_value  max_value                  timestamp
  1 test-session-1751138409 test_slider    0.5        0.0        1.0 2025-06-28T15:20:09.151421

🆔 Unique sessions: 1
   - test-session-1751138409

🎛️ Unique sliders: 1
   - test_slider

⚙️ SLIDER_CONFIGS TABLE
-------------------------
Columns:
   - id (INTEGER)
   - name (TEXT)
   - t

In [81]:
# create_missing_tables.py - Run this to create the missing tables

import sqlite3
import os
from datetime import datetime

# Database configuration
DB_PATH = "schedules.db"  # Update this if your database is in a different location

def check_existing_tables():
    """Check what tables currently exist"""
    print("🔍 Checking existing tables...")
    
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Get list of all tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        print(f"📊 Found {len(tables)} existing tables:")
        for table in tables:
            print(f"   - {table[0]}")
        
        # Check specifically for our tables
        table_names = [table[0] for table in tables]
        has_configs = 'slider_configs' in table_names
        has_recordings = 'slider_recordings' in table_names
        
        print(f"\n🎚️ slider_configs table: {'✅ EXISTS' if has_configs else '❌ MISSING'}")
        print(f"🎚️ slider_recordings table: {'✅ EXISTS' if has_recordings else '❌ MISSING'}")
        
        conn.close()
        return has_configs, has_recordings
        
    except Exception as e:
        print(f"❌ Error checking tables: {e}")
        return False, False

def create_slider_tables():
    """Create the missing slider tables"""
    print("\n🔧 Creating missing slider tables...")
    
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Create slider_configs table
        print("📝 Creating slider_configs table...")
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS slider_configs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            thresholds TEXT NOT NULL,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        # Create slider_recordings table
        print("📝 Creating slider_recordings table...")
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS slider_recordings (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            session_id TEXT,
            slider_key TEXT NOT NULL,
            value REAL NOT NULL,
            min_value REAL NOT NULL,
            max_value REAL NOT NULL,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        # Create indexes for better performance
        print("📝 Creating indexes...")
        cursor.execute("""
        CREATE INDEX IF NOT EXISTS idx_slider_session_id 
        ON slider_recordings(session_id)
        """)
        
        cursor.execute("""
        CREATE INDEX IF NOT EXISTS idx_slider_key 
        ON slider_recordings(slider_key)
        """)
        
        cursor.execute("""
        CREATE INDEX IF NOT EXISTS idx_slider_timestamp 
        ON slider_recordings(timestamp)
        """)
        
        # Commit changes
        conn.commit()
        print("✅ Tables created successfully!")
        
        # Verify tables were created
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE 'slider_%';")
        slider_tables = cursor.fetchall()
        print(f"✅ Verified: {len(slider_tables)} slider tables now exist")
        
        conn.close()
        return True
        
    except Exception as e:
        print(f"❌ Error creating tables: {e}")
        return False

def test_table_creation():
    """Test that we can insert data into the new tables"""
    print("\n🧪 Testing table creation...")
    
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Test slider_recordings table
        print("🧪 Testing slider_recordings insert...")
        cursor.execute("""
        INSERT INTO slider_recordings (session_id, slider_key, value, min_value, max_value)
        VALUES (?, ?, ?, ?, ?)
        """, ('test-creation', 'test_key', 0.5, 0.0, 1.0))
        
        # Test slider_configs table
        print("🧪 Testing slider_configs insert...")
        cursor.execute("""
        INSERT INTO slider_configs (name, thresholds)
        VALUES (?, ?)
        """, ('test-config', '{"test": 0.5}'))
        
        conn.commit()
        
        # Verify data was inserted
        cursor.execute("SELECT COUNT(*) FROM slider_recordings WHERE session_id = 'test-creation'")
        recording_count = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(*) FROM slider_configs WHERE name = 'test-config'")
        config_count = cursor.fetchone()[0]
        
        print(f"✅ Test records inserted: {recording_count} recording, {config_count} config")
        
        # Clean up test data
        cursor.execute("DELETE FROM slider_recordings WHERE session_id = 'test-creation'")
        cursor.execute("DELETE FROM slider_configs WHERE name = 'test-config'")
        conn.commit()
        
        conn.close()
        print("✅ Table tests passed!")
        return True
        
    except Exception as e:
        print(f"❌ Table test failed: {e}")
        return False

def show_table_schema():
    """Show the schema of the created tables"""
    print("\n📋 Table schemas:")
    
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Show slider_recordings schema
        cursor.execute("PRAGMA table_info(slider_recordings)")
        columns = cursor.fetchall()
        print("\n🎚️ slider_recordings columns:")
        for col in columns:
            print(f"   - {col[1]} ({col[2]}) {'PRIMARY KEY' if col[5] else ''}")
        
        # Show slider_configs schema
        cursor.execute("PRAGMA table_info(slider_configs)")
        columns = cursor.fetchall()
        print("\n⚙️ slider_configs columns:")
        for col in columns:
            print(f"   - {col[1]} ({col[2]}) {'PRIMARY KEY' if col[5] else ''}")
        
        conn.close()
        
    except Exception as e:
        print(f"❌ Error showing schema: {e}")

if __name__ == "__main__":
    print("🔧 SLIDER TABLES CREATION TOOL")
    print("="*50)
    print(f"📁 Database: {DB_PATH}")
    print(f"⏰ Time: {datetime.now()}")
    
    # Check if database file exists
    if not os.path.exists(DB_PATH):
        print(f"❌ Database file {DB_PATH} not found!")
        print("   Make sure you're running this from the correct directory")
        exit(1)
    
    # Check existing tables
    has_configs, has_recordings = check_existing_tables()
    
    # Create missing tables
    if not has_configs or not has_recordings:
        success = create_slider_tables()
        if not success:
            print("❌ Failed to create tables")
            exit(1)
    else:
        print("✅ All slider tables already exist!")
    
    # Test the tables
    test_success = test_table_creation()
    if not test_success:
        print("❌ Table tests failed")
        exit(1)
    
    # Show final schema
    show_table_schema()
    
    print("\n🎉 SUCCESS!")
    print("="*30)
    print("✅ All slider tables are ready")
    print("✅ Tables tested successfully")
    print("🚀 Your Flask app should now work!")
    print("\nNext steps:")
    print("1. Restart your Flask app (python run.py)")
    print("2. Test the slider recording in your React app")
    print("3. Run the isolation test again to verify")

🔧 SLIDER TABLES CREATION TOOL
📁 Database: schedules.db
⏰ Time: 2025-07-02 20:45:46.498537
🔍 Checking existing tables...
📊 Found 0 existing tables:

🎚️ slider_configs table: ❌ MISSING
🎚️ slider_recordings table: ❌ MISSING

🔧 Creating missing slider tables...
📝 Creating slider_configs table...
📝 Creating slider_recordings table...
📝 Creating indexes...
✅ Tables created successfully!
✅ Verified: 2 slider tables now exist

🧪 Testing table creation...
🧪 Testing slider_recordings insert...
🧪 Testing slider_configs insert...
✅ Test records inserted: 1 recording, 1 config
✅ Table tests passed!

📋 Table schemas:

🎚️ slider_recordings columns:
   - id (INTEGER) PRIMARY KEY
   - session_id (TEXT) 
   - slider_key (TEXT) 
   - value (REAL) 
   - min_value (REAL) 
   - max_value (REAL) 
   - timestamp (TIMESTAMP) 

⚙️ slider_configs columns:
   - id (INTEGER) PRIMARY KEY
   - name (TEXT) 
   - thresholds (TEXT) 
   - timestamp (TIMESTAMP) 

🎉 SUCCESS!
✅ All slider tables are ready
✅ Tables tested s